In [ ]:
import os
import re
import random
from collections import defaultdict
from itertools import combinations
from pathlib import Path
import string

def generar_txts(carpeta, cantidad=100):
    os.makedirs(carpeta, exist_ok=True)
    palabras_base = [
        #usamos 25 palabras
    "estudiante", "universidad", "tarea", "plagio", "docente", "investigación", "ciencia",
    "aprendizaje", "conocimiento", "disciplina", "método", "estudio", "ensayo", "análisis", "criterio",
    "creatividad", "descubrimiento", "innovación", "teoría",]
    for i in range(cantidad):
        #la longitud es de 200 palabras
        contenido = " ".join(random.choices(palabras_base, k=200))
        with open(os.path.join(carpeta, f"documento_{i+1}.txt"), "w", encoding="utf-8") as f:
            f.write(contenido)
            

def cargar_documentos(ruta, n=2):
    documentos = {}
    for archivo in os.listdir(ruta):
        if archivo.endswith(".txt"):
            with open(os.path.join(ruta, archivo), "r", encoding="utf-8") as f:
                contenido = f.read()  # Aseguramos que el contenido sea un string
                documentos[archivo] = preprocesar_texto(contenido, n)  # Procesamos el texto en lugar de una lista
    return documentos

def preprocesar_texto(texto, n=2):
    if isinstance(texto, list):  # Si el texto ya es una lista, convertimos de nuevo en texto
        texto = " ".join(texto)
    texto = texto.lower()
    texto = texto.translate(str.maketrans("", "", string.punctuation))
    palabras = texto.split()
    ngramas = [" ".join(palabras[i:i+n]) for i in range(len(palabras)-n+1)]
    return ngramas


def construir_tabla_hash(documentos, n=3):
    tablas = {}
    for nombre, texto in documentos.items():
        ngramas = preprocesar_texto(texto, n)
        # Utilizamos un set para almacenar los n-gramas únicos de cada documento
        tabla_hash = set(hash(ng) for ng in ngramas)
        tablas[nombre] = tabla_hash
    return tablas


def similitud_jaccard(set1, set2):
    interseccion = len(set1 & set2)
    union = len(set1 | set2)
    return interseccion / union if union != 0 else 0

def comparar_documentos(tablas):
    #Se compararán 4,950 pares (100 * 99 / 2).
    similitudes = []
    for doc1, doc2 in combinations(tablas.keys(), 2):
        sim = similitud_jaccard(tablas[doc1], tablas[doc2])
        similitudes.append((doc1, doc2, sim))
    return similitudes


def merge_sort(lista):
    if len(lista) > 1:
        mid = len(lista) // 2
        izquierda = lista[:mid]
        derecha = lista[mid:]
        
        merge_sort(izquierda)
        merge_sort(derecha)
        
        i = j = k = 0
        while i < len(izquierda) and j < len(derecha):
            if izquierda[i][2] > derecha[j][2]:
                lista[k] = izquierda[i]
                i += 1
            else:
                lista[k] = derecha[j]
                j += 1
            k += 1
        
        while i < len(izquierda):
            lista[k] = izquierda[i]
            i += 1
            k += 1
        
        while j < len(derecha):
            lista[k] = derecha[j]
            j += 1
            k += 1
            
def mostrar_top_similares(similitudes, top_n=10):
    merge_sort(similitudes)
    print("Top", top_n, "documentos más similares:")
    for i in range(min(top_n, len(similitudes))):
        print(f"{similitudes[i][0]} - {similitudes[i][1]}: {similitudes[i][2]*10:.5f} %")

#usamos una ruta dentro del mismo ambiente
ruta_escritorio = Path.cwd().parent.parent / "documentos"

# Generar archivos de prueba
generar_txts(ruta_escritorio, 100)

# Procesar documentos
documentos = cargar_documentos(ruta_escritorio)
tablas_hash = construir_tabla_hash(documentos)
similitudes = comparar_documentos(tablas_hash)
mostrar_top_similares(similitudes, 10)


Top 10 documentos más similares:
documento_49.txt - documento_82.txt: 3.78685 %
documento_4.txt - documento_93.txt: 3.61364 %
documento_19.txt - documento_68.txt: 3.58862 %
documento_76.txt - documento_99.txt: 3.57477 %
documento_19.txt - documento_52.txt: 3.55180 %
documento_14.txt - documento_88.txt: 3.54402 %
documento_38.txt - documento_56.txt: 3.52144 %
documento_26.txt - documento_65.txt: 3.51770 %
documento_23.txt - documento_52.txt: 3.48936 %
documento_53.txt - documento_85.txt: 3.48584 %


In [ ]:
ruta_documentos = Path(__file__).resolve().parent.parent / "documentos"
otra_opcion = ruta_escritorio = Path.cwd() / "documentos"